In [7]:
#import libraries 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [22]:
file_path_metadata = '/Users/renata.m/QIIME2/qiime2-final_analysis-silva_abx/sra-metadata.tsv'
sra_metadata = pd.read_csv(file_path_metadata, sep='\t')
sra_metadata = sra_metadata[['ID']]
sra_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494 entries, 0 to 493
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      494 non-null    object
dtypes: object(1)
memory usage: 4.0+ KB


In [23]:
file_path_sample_data = '/Users/renata.m/Desktop/Thesis/Rashidi_Data_2022/StoolSamplesTable.csv'
sample_df = pd.read_csv(file_path_sample_data, header=2)
sample_df = sample_df.rename({'Accession_Number':'ID'}, axis=1)
sample_df = sample_df[['ID', 'Patient_ID', 'SampleDayRelativeToD1Chemo']]
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 566 entries, 0 to 565
Data columns (total 3 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ID                          566 non-null    object
 1   Patient_ID                  566 non-null    object
 2   SampleDayRelativeToD1Chemo  566 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 13.4+ KB


In [24]:
sample_sra_metadata = sample_df.merge(sra_metadata, on='ID', how='inner')
sample_sra_metadata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494 entries, 0 to 493
Data columns (total 3 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   ID                          494 non-null    object
 1   Patient_ID                  494 non-null    object
 2   SampleDayRelativeToD1Chemo  494 non-null    int64 
dtypes: int64(1), object(2)
memory usage: 11.7+ KB


In [25]:
file_path_patient_data = '/Users/renata.m/Desktop/Thesis/Rashidi_Data_2022/PatientMetadata.csv'
patient_df = pd.read_csv(file_path_patient_data, header = 5)
patient_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68 entries, 0 to 67
Data columns (total 9 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Patient_ID                         68 non-null     object 
 1   Age                                68 non-null     int64  
 2   Sex                                68 non-null     object 
 3   Disease_Phase                      68 non-null     object 
 4   Chemotherapy                       68 non-null     object 
 5   FirstNFDayRelativeToD1Chemo        64 non-null     float64
 6   DiarrheaStartDayRelativeToD1Chemo  36 non-null     float64
 7   DiarrheaEndDayRelativeToD1Chemo    36 non-null     float64
 8   FirstCDiffDayRelativeToD1Chemo     17 non-null     float64
dtypes: float64(4), int64(1), object(4)
memory usage: 4.9+ KB


In [26]:
sra_metadata_patient = sample_sra_metadata.merge(patient_df, on=['Patient_ID'], how='inner')
sra_metadata_patient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494 entries, 0 to 493
Data columns (total 11 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   ID                                 494 non-null    object 
 1   Patient_ID                         494 non-null    object 
 2   SampleDayRelativeToD1Chemo         494 non-null    int64  
 3   Age                                494 non-null    int64  
 4   Sex                                494 non-null    object 
 5   Disease_Phase                      494 non-null    object 
 6   Chemotherapy                       494 non-null    object 
 7   FirstNFDayRelativeToD1Chemo        473 non-null    float64
 8   DiarrheaStartDayRelativeToD1Chemo  289 non-null    float64
 9   DiarrheaEndDayRelativeToD1Chemo    289 non-null    float64
 10  FirstCDiffDayRelativeToD1Chemo     176 non-null    float64
dtypes: float64(4), int64(2), object(5)
memory usage: 42.6+ KB


In [27]:
# Create binary column for CDiff infection
sra_metadata_patient['CDiff_infection'] = sra_metadata_patient['FirstCDiffDayRelativeToD1Chemo'].notna().astype(int)

# Create binary column for NF
sra_metadata_patient['NF'] = sra_metadata_patient['FirstNFDayRelativeToD1Chemo'].notna().astype(int)

# Create binary column for NF
sra_metadata_patient['Diarrhea'] = sra_metadata_patient['DiarrheaStartDayRelativeToD1Chemo'].notna().astype(int)

In [28]:
#bins and labels for the time 
bins = [-float('inf'), -1, 4, 9, 15, 21, 28,float('inf')]
labels = ['0_Before D1 chemo', '1_Day 0 to 4', '2_Day 5 to 9', '3_Day 10 to 15', '4_Day 16 to 22', '5_Day 23 to 28', '6_Day 28+']

# new column 'Time_group'
sra_metadata_patient_time = sra_metadata_patient.assign(Time_group=pd.cut(sra_metadata_patient['SampleDayRelativeToD1Chemo'], bins=bins, labels=labels))


In [29]:
sra_metadata_patient_time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 494 entries, 0 to 493
Data columns (total 15 columns):
 #   Column                             Non-Null Count  Dtype   
---  ------                             --------------  -----   
 0   ID                                 494 non-null    object  
 1   Patient_ID                         494 non-null    object  
 2   SampleDayRelativeToD1Chemo         494 non-null    int64   
 3   Age                                494 non-null    int64   
 4   Sex                                494 non-null    object  
 5   Disease_Phase                      494 non-null    object  
 6   Chemotherapy                       494 non-null    object  
 7   FirstNFDayRelativeToD1Chemo        473 non-null    float64 
 8   DiarrheaStartDayRelativeToD1Chemo  289 non-null    float64 
 9   DiarrheaEndDayRelativeToD1Chemo    289 non-null    float64 
 10  FirstCDiffDayRelativeToD1Chemo     176 non-null    float64 
 11  CDiff_infection                    494 non-nu

In [30]:
#sra_metadata_patient_time.to_csv('/Users/renata.m/QIIME2/qiime2-final_analysis-silva_abx/sra_metadata_time.csv')